<a href="https://colab.research.google.com/github/mgmills93/AEM_Spring_2019/blob/master/THA4/Take_Home_Assignment_4_Mary_Mills.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

jail_data = pd.read_csv('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/THA4/final_ASJ15_set.csv')
pop_data = pd.read_csv('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/THA4/ACS_15_5YR_B03001_with_ann_copy.csv', header=1)
sc_arrests = pd.read_excel('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/enforcement/Secure_Communities_2017_Deportations.xlsx', header=4)

In [0]:
def filter_2015(sc_arrests):
  sc_arrests2015 = sc_arrests[sc_arrests['Departed Date'].isin(pd.date_range(start='2015-01-01', end='2015-12-31'))]
  
  return sc_arrests2015

In [0]:
sc_arrests2015 = filter_2015(sc_arrests)
def sum_arrests2015(sc_arrests2015):
  sum_arrests = sc_arrests2015.groupby('State').size().to_frame('Arrest_Number').reset_index()
  
  return sum_arrests

In [0]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',}



In [0]:
def clean_pop(pop_data):
  pop_columns = pop_data[['Geography', 'Estimate; Total:', 'Estimate; Total: - Hispanic or Latino:']]
  pop_columns.rename(columns={'Geography':'State', 'Estimate; Total:':'Total_Pop', 'Estimate; Total: - Hispanic or Latino:':'Hisp_Pop'}, inplace=True)
  pop_columns['Percent_Hispanic'] = pop_columns['Hisp_Pop']/pop_columns['Total_Pop']*100
  pop_columns['State'] = pop_columns['State'].map(us_state_abbrev)
  
  return pop_columns

In [0]:
missing_states = [pd.Series(['Alaska', np.nan, np.nan, np.nan], index=cols_jail.columns), 
                  pd.Series(['Connecticut', np.nan, np.nan, np.nan], index=cols_jail.columns), 
                  pd.Series(['Delaware', np.nan, np.nan, np.nan], index=cols_jail.columns), 
                  pd.Series(['Hawaii', np.nan, np.nan, np.nan], index=cols_jail.columns),
                  pd.Series(['Rhode Island', np.nan, np.nan, np.nan], index=cols_jail.columns), 
                  pd.Series(['Vermont', np.nan, np.nan, np.nan], index=cols_jail.columns)]

def jail_fill(jail_data):
  cols_jail_total = jail_data.append(missing_states, ignore_index=True)
  cols_jail_total.mean()
  cols_jail_filled = cols_jail_total.fillna(cols_jail_total.mean())

  return cols_jail_filled

In [0]:
cols_jail_filled = jail_fill(jail_data)
def clean_jail(cols_jail_filled):
  final_jail = cols_jail_filled.drop(columns='Unnamed: 0')
  final_jail['Percent_NONCITZ'] = final_jail['NONCITZ']/final_jail['TOTPOP']*100
  final_jail.iloc[5, final_jail.columns.get_loc('State')] = "District of Columbia"
  final_jail.iloc[37, final_jail.columns.get_loc('State')] = "Tennessee"
  final_jail['State'] = final_jail['State'].map(us_state_abbrev)
  
  return final_jail

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [0]:
final_jail = clean_jail(cols_jail_filled)
final_pop = clean_pop(pop_data)

def pop_jail_merge(final_jail):
  pop_jail = final_jail.merge(final_pop, on='State', how='inner')
  
  return pop_jail

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

In [0]:
sum_arrests = sum_arrests2015(sc_arrests2015)
pop_jail = pop_jail_merge(final_jail)

def final_merge(sum_arrests):
  final_merge = sum_arrests.merge(pop_jail, on='State', how='inner')
  
  return final_merge

###Pipeline

In [170]:
#filter sc_arrests for only 2015 deportations
filter_2015(sc_arrests)
#group 2015 deportations by state
sum_arrests2015(sc_arrests2015)
#clean population data and add Percent_Hispanic
clean_pop(pop_data)
#add missing states to jail dataset and fill values with means
jail_fill(jail_data)
#clean jail data and add Percent_NONCITZ
clean_jail(cols_jail_filled)
#merge population and jail data
pop_jail_merge(final_jail)
#merge pop_jail with SC arrest dataset
final_merge(sum_arrests)
#save merged dataset
final_merge(sum_arrests).to_csv('final_tha4_set.csv')

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

,State,Arrest_Number,NONCITZ,Percent_NONCITZ,TOTPOP,Total_Pop,Hisp_Pop,Percent_Hispanic
0,AK,3,277.755556,2.393353,11605.288889,733375,47808,6.518902
1,AL,440,340.000000,5.432178,6259.000000,4830620,193492,4.005531
2,AR,350,88.000000,2.771654,3175.000000,2958208,203226,6.869902
3,AZ,4685,256.000000,2.388951,10716.000000,6641928,2014711,30.333226
4,CA,9827,1384.000000,1.882635,73514.000000,38421464,14750686,38.391785
5,CO,901,327.000000,3.702026,8833.000000,5278906,1112586,21.076071
6,CT,158,277.755556,2.393353,11605.288889,3593222,526508,14.652810
7,DC,63,53.000000,2.866414,1849.000000,647484,65803,10.162877
8,DE,75,277.755556,2.393353,11605.288889,926454,80706,8.711280
9,FL,1824,903.000000,1.914109,47176.000000,19645772,4660733,23.723848
